In [1]:
import requests
import json
import pandas as pd
import os
from concurrent.futures import ThreadPoolExecutor

In [11]:
def get_ids(search_term: str, limit: str = 50):
    '''
    Função responsável por por listar os ids da pesquisa
    '''

    # Monta a URL
    url = f"https://api.mercadolibre.com/sites/MLA/search?q={search_term}&limit={limit}#json"
    # Usa a biblioteca requests para fazer a requisição e transformar os resultados em um objeto json
    response_json = requests.get(url).json()
    # Itera sobre o json para armazenar os ids em uma lista
    ids = list()
    for response in response_json['results']:
        ids.append(response['id'])

    return ids

In [4]:
def extrai_array(atributos: str, id: str):
    '''
    Função que extrai atributos dos campos de lista
    '''
    for atributo in atributos:
        if atributo.get('id') == id:
            return atributo.get('value_name')

In [15]:
def get_info(id: str):
    '''
    Função responsável por consultar resultados do produto pelo id
    e extrair informações relevantes do json
    '''

    # Consulta API para retornar resultados de acordo com o ID
    print('Consultando resultados da API para o id:', id)
    url = f'https://api.mercadolibre.com/items/{id}'
    print('Consultando informações da URL:', url)
    results = requests.get(url).json()

    # Filtrando as colunas relevantes
    print('Filtrando resultados...')
    filtered_results = {
        'id': results.get('id'),
        'site_id': results.get('site_id'),
        'title': results.get('title'),
        'seller_id': results.get('seller_id'),
        'category_id': results.get('category_id'),
        'price': results.get('price'),
        'original_price': results.get('original_price'),
        'currency_id': results.get('currency_id'),
        'initial_quantity': results.get('initial_quantity'),
        'accepts_mercadopago': results.get('accepts_mercadopago'),
        'status': results.get('status'),
        'shipping_local_pick_up': results.get('shipping', {}).get('local_pick_up'),
        'shipping_free_shipping': results.get('shipping', {}).get('free_shipping'),
        'seller_address_state': results.get('seller_address', {}).get('state', {}).get('name'),
        'seller_address_country': results.get('seller_address', {}).get('country', {}).get('name'),
        'warranty_time': extrai_array(results.get('sale_terms', []), 'WARRANTY_TIME'),
        'warranty_type': extrai_array(results.get('sale_terms', []), 'WARRANTY_TYPE'),
        'brand': extrai_array(results.get('attributes', []), 'BRAND'),
        'color': extrai_array(results.get('attributes', []), 'COLOR'),
        'depth': extrai_array(results.get('attributes', []), 'DEPTH'),
        'diameter': extrai_array(results.get('attributes', []), 'DIAMETER'),
        'functions': extrai_array(results.get('attributes', []), 'FUNCTIONS'),
        'height': extrai_array(results.get('attributes', []), 'HEIGHT'),
        'is_moisture_resistant': extrai_array(results.get('attributes', []), 'IS_MOISTURE_RESISTANT'),
        'item_condition': extrai_array(results.get('attributes', []), 'ITEM_CONDITION'),
        'manufacturer': extrai_array(results.get('attributes', []), 'MANUFACTURER'),
        'model': extrai_array(results.get('attributes', []), 'MODEL'),
        'ram_memory': extrai_array(results.get('attributes', []), 'RAM_MEMORY'),
        'operative_system': extrai_array(results.get('attributes', []), 'OPERATING_SYSTEM'),
        'operative_systems_compatibility': extrai_array(results.get('attributes', []), 'OPERATIVE_SYSTEMS_COMPATIBILITY'),
        'with_integrated_voice_assistants': extrai_array(results.get('attributes', []), 'WITH_INTEGRATED_VOICE_ASSISTANTS'),
        'virtual_assistants': extrai_array(results.get('attributes', []), 'VIRTUAL_ASSISTANTS'),
        'storage_capacity': extrai_array(results.get('attributes', []), 'STORAGE_CAPACITY'),
        'voltage': extrai_array(results.get('attributes', []), 'VOLTAGE'),
        'weight': extrai_array(results.get('attributes', []), 'WEIGHT'),
        'width': extrai_array(results.get('attributes', []), 'WIDTH'),
        'with_360_sound': extrai_array(results.get('attributes', []), 'WITH_360_SOUND'),
        'with_bluetooth': extrai_array(results.get('attributes', []), 'WITH_BLUETOOTH'),
        'with_camera': extrai_array(results.get('attributes', []), 'WITH_CAMERA'),
        'with_digital_display': extrai_array(results.get('attributes', []), 'WITH_DIGITAL_DISPLAY'),
        'with_recyclable_materials': extrai_array(results.get('attributes', []), 'WITH_RECYCLABLE_MATERIALS'),
        'with_wifi_connection': extrai_array(results.get('attributes', []), 'WITH_WI_FI_CONNECTION'),
        'with_wifi': extrai_array(results.get('attributes', []), 'WITH_WI_FI')
    }

    return filtered_results


In [16]:
def write_to_csv(data, csv_file):
    # Verifica se o arquivo CSV já existe
    if os.path.exists(csv_file):
        # Carrega o CSV existente
        existing_data = pd.read_csv(csv_file)
        
        # Converte os dados existentes para um DataFrame
        existing_ids = existing_data['id'].tolist()
        
        # Filtra os novos dados para remover entradas com IDs já existentes
        new_data = [entry for entry in data if entry['id'] not in existing_ids]
    else:
        new_data = data

    if not new_data:
        print('Sem dados para inserir.')

    # Converte a nova lista de dicionários para um DataFrame
    df = pd.DataFrame(new_data)

    # Salva o DataFrame no arquivo CSV
    print('Salvando arquivo...')
    df.to_csv(csv_file, mode='a', index=False, header=not os.path.exists(csv_file))

In [17]:
search_terms = ['chromecast', 'Google Home', 'Apple TV', 'Amazon Fire TV']
# Itera sobre os termos de busca
for search_term in search_terms:
    ids = get_ids(search_term)

    # Usando threads, é possível melhorar a performance com paralelismo
    # Diminuição notada nos testes (50 itens): 20s para 5s 
    with ThreadPoolExecutor(max_workers=5) as executor:
        results = executor.map(get_info, ids)
    
    csv_file = 'data/mercado_livre.csv'
    os.makedirs('data', exist_ok=True)  # Cria o diretório se não existir
    write_to_csv(results, csv_file)

Consultando resultados da API para o id:Consultando resultados da API para o id: MLA1520563158
Consultando informações da URL: https://api.mercadolibre.com/items/MLA1520563158
 MLA1434031785
Consultando informações da URL: https://api.mercadolibre.com/items/MLA1434031785
Consultando resultados da API para o id: MLA1876051918
Consultando informações da URL: https://api.mercadolibre.com/items/MLA1876051918
Consultando resultados da API para o id: MLA1418248629
Consultando informações da URL: https://api.mercadolibre.com/items/MLA1418248629
Consultando resultados da API para o id: MLA1874224766
Consultando informações da URL: https://api.mercadolibre.com/items/MLA1874224766
Filtrando resultados...
Consultando resultados da API para o id: MLA1108749509
Consultando informações da URL: https://api.mercadolibre.com/items/MLA1108749509
Filtrando resultados...
Consultando resultados da API para o id: MLA1887937376
Consultando informações da URL: https://api.mercadolibre.com/items/MLA1887937376
